In [ ]:
# SHARP

# blur = cv2.GaussianBlur(img,(5, 5),0).astype(int)
# sub = img.astype(int) - blur.astype(int)
# sharp = np.clip(img.astype(int) + sub, a_min = 0, a_max = 255).astype('uint8')
# sharp2 = np.clip(img.astype(int) + sub*2, a_min = 0, a_max = 255).astype('uint8')


# cv2.imshow('ori', img.astype('uint8'))
# cv2.imshow('sharp', sharp.astype('uint8'))
# cv2.imshow('sharp2', sharp2.astype('uint8'))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Find corner

In [ ]:
fig , axs = plt.subplots(2, 1, figsize=(18, 20), dpi=60)
fig.subplots_adjust(hspace=0.1, wspace=0.1) #設定子圖的間隔
fig.suptitle('Corner Detect', y=0.92, fontsize=32)

# sharpen
blur = cv2.GaussianBlur(img,(5, 5),0).astype(int)
sub = img.astype(int) - blur
sharped_img = np.clip(img.astype(int) + sub*5, a_min = 0, a_max = 255).astype('uint8')

# sharped = cv2.filter2D(gray, -1, kernel)
sharped_img = adjust_gamma(sharped_img, gamma)
# sharped_img = cv2.GaussianBlur(sharped_img,(5, 5),0)
sharped = cv2.cvtColor(sharped_img, cv2.COLOR_BGR2GRAY).astype(np.float32) # better

# find Harris corners
blockSize = 5 # 越大越不注意細節，只憑個大概位置
ksize = 3 # 越大越敏感，但是位置越不精確(特別是模糊地帶)
k = 0.04 if False else 0.06
dst = cv2.cornerHarris(sharped,blockSize,ksize,k)
# dst = cv2.dilate(dst,None)
ret, dst = cv2.threshold(dst,0.05*dst.max(),255,0)
# find centroids
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst.astype(np.uint8))

# define the criteria to stop and refine the corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(gray,np.float32(centroids),(1,1),(-1,-1),criteria)

# Now draw them
# res = np.hstack((centroids,corners))
# res = np.int0(res)
res = np.where(dst>0)
print(res)

mark = np.zeros_like(img)
# use_centroids = False
# if use_centroids:
#     mark[res[:,1],res[:,0]]   = [255,0,0]
#     mark[res[:,1]+1,res[:,0]] = [255,0,0]
#     mark[res[:,1]-1,res[:,0]] = [255,0,0]
#     mark[res[:,1],res[:,0]+1] = [255,0,0]
#     mark[res[:,1],res[:,0]-1] = [255,0,0]
#     mask2 = mark[:,:,0] != 0
#     mask2 = np.stack((mask2,mask2,mask2), 2)
# else:
#     mark[res[:,3],res[:,2]]   = [0,0,255]
#     mark[res[:,3]+1,res[:,2]] = [0,0,255]
#     mark[res[:,3]-1,res[:,2]] = [0,0,255]
#     mark[res[:,3],res[:,2]+1] = [0,0,255]
#     mark[res[:,3],res[:,2]-1] = [0,0,255]
#     mask2 = mark[:,:,2] != 0
#     mask2 = np.stack((mask2,mask2,mask2), 2)

mark[res[0],res[1]] = [255,0,0]
# mark[res[0]+1,res[1]] = [255,0,0]
# mark[res[0]-1,res[1]] = [255,0,0]
# mark[res[0],res[1]+1] = [255,0,0]
# mark[res[0],res[1]-1] = [255,0,0]
mask2 = mark[:,:,0] != 0
mask2 = np.stack((mask2,mask2,mask2), 2)
    
mask2 = mask2*mask

# axs[0].imshow(cv2.cvtColor(mark, cv2.COLOR_BGR2RGB))
# axs[0].set_title('Corners', fontsize=24)

mark = mark*mask

axs[0].imshow(mark)
axs[0].set_title('Corners Masked', fontsize=24)

points = np.where(mark*(mask2!=0)!=0)
img_corner = img*(mask2==0) + cv2.cvtColor(mark*(mask2!=0), cv2.COLOR_BGR2RGB)

axs[1].imshow(cv2.cvtColor(img_corner, cv2.COLOR_BGR2RGB))
axs[1].set_title('Corners on image', fontsize=24)
plt.show()

# cv2.imshow('123', img_corner.astype('uint8'))
# cv2.imshow('1234', sharped.astype('uint8'))
# cv2.waitKey(0)
# cv2.destroyAllWindows()
print(len(points[0]), 'Points')

# Draw path & Count

In [ ]:
# e_m_lines = [l if not (l==np.array([0,0,0,0])).all() else pass for l in e_m_lines]
p_score = np.zeros_like(mask[...,0], int)
tmp3 = np.zeros_like(p_score, 'uint8')

# for x1,y1,x2,y2 in e_m_lines:
#     edge_p1, edge_p2 = find_edge(np.array([x1,y1]), np.array([x2,y2]), p_score.shape[0], p_score.shape[1])
#     full_ex_line = interpolate(np.concatenate((edge_p1, edge_p2), 0))
#     p_score[full_ex_line[:,1], full_ex_line[:,0]] = p_score[full_ex_line[:,1], full_ex_line[:,0]] + 1 #vote
#     # intersections
#     cv2.line(tmp3, (x1,y1), (x2,y2), (25,25,25),1)

for l in fake_sets:
    for x1,y1,x2,y2 in l:
        edge_p1, edge_p2 = find_edge(np.array([x1,y1]), np.array([x2,y2]), p_score.shape[0], p_score.shape[1])
        full_ex_line = interpolate(np.concatenate((edge_p1, edge_p2), 0))
        p_score[full_ex_line[:,1], full_ex_line[:,0]] = p_score[full_ex_line[:,1], full_ex_line[:,0]] + 1 #vote
        # intersections
        cv2.line(tmp3, (x1,y1), (x2,y2), (25,25,25),1)
    
# Intersections
intersections = {}
for p,l1 in enumerate(e_m_lines):
    if (l1==0).all():
        continue
    for q,l2 in enumerate(e_m_lines[p+1:]):
        if (l2==0).all():
            continue
        X,Y = find_intersection(l1, l2)
        if X>=0 and Y>=0 and X<_w and Y<_h:
            intersections[str(p)+'_'+str(q)] = np.array([X,Y])
    
# mark*(mask2!=0)
p_score = p_score * ((mark*(mask2!=0))[...,0] != 0)
fig = plt.figure(figsize=(24, 12), dpi=60)
fig.suptitle('Court point', y=0.92, fontsize=32)
        
rank = np.unique(p_score)
print(rank)
plt.imshow((p_score>=2).astype('uint8')*205 + tmp3, cmap='gray', vmin=0, vmax=255)
plt.show()

cv2.imwrite('corners.png', (p_score>=2).astype('uint8')*205 + tmp3)
# cv2.imshow('1234', (p_score>=2).astype('uint8')*205 + tmp3)
# cv2.waitKey(0)
# cv2.destroyAllWindows()